In [4]:
from langchain_community.vectorstores import FAISS
import faiss
from langchain_core.documents import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
from ingestor.ordine_ingestor import OrderIngestor
from ingestor.menu_ingestor import MenuIngestor
from ingestor.chunk import Chunk
from ingestor.config import EmbedderConf
from ingestor.embedder import ChunkEmbedder

In [5]:
menu_ingestor = MenuIngestor(llm=None)

chunks_menu = menu_ingestor.chunks_from_docs(base_dir="HackapizzaDataset/Menu")

chunks_menu

[Chunk(id='7752a9ff-110a-5090-b735-c8e8d23c35af', filename='L Essenza di Asgard.pdf', text='<h1>Ristorante: L\'Essenza di Asgard</h1> Chef Palissandro "Sandro" Luminetti Nel regno scintillante di Asgard, dove il Bifrost collega mondi lontani e il cielo racconta storie di eroi e dèi, lo Chef Palissandro "Sandro" Luminetti emerge come una forza divina della creazione culinaria. Ogni giorno, il suo talento esplode come un\'enorme supernova, trasformando la cucina in una danza di energia cosmica. Non è raro vederlo in una contemplazione profonda davanti ai suoi mistici fornelli multicelesti, mentre il suo spirito esplora infinite realtà parallele vantando di un LTK ben oltre superiore al VI, plasmando le sue creazioni in delicate sinfonie di essenze primarie. La sua arte culinaria si fonda su una connessione psichica certificata di grado II con gli ingredienti più rari e mitici. Le sue mani si muovono con una grazia eterea, orchestrando dialoghi silenziosi tra spezie elfiche, frutti delle 

In [6]:
pdf_path = "HackapizzaDataset/Misc/Manuale di Cucina.pdf"

order_ingestor = OrderIngestor()

chunks_ordini = order_ingestor.chunks_from_doc(pdf_path)

chunks_ordini

[Chunk(id='420c7326-8d9a-577f-8632-506600f96b87', filename='Manuale di Cucina.pdf', text='<h1>Capitolo 2: Gli Ordini</h1> Qui di seguito vi illustro i diversi ordini presenti nel nostro universo. FONDAMENTALE leggere.', embedding=None, embeddings_model=None),
 Chunk(id='d9cb75fd-d3b0-56de-a5ba-83c87a85517a', filename='Manuale di Cucina.pdf', text="<h3>🪐 Ordine della Galassia di Andromeda</h3> Cari pionieri del gusto cosmico, preparatevi a una sfida da veri fuoriclasse: cucinare per gli abitanti di Andromeda! Qui non parliamo solo di tecnica stellare, ma di pura diplomazia culinaria intergalattica. La particolarita? Ogni piatto dev'essere rigorosamente privo di lattosio - non un goccio, nemmeno l'ombra di una molecola originaria della Via Lattea. Se pensate che il formaggio sia l'unica via per la felicita, e tempo di ripensare il vostro approccio. Imparerete a creare sapori complessi senza latte, burro o panna, conquistando palati alieni con alternative che sfidano le leggi della gastro

In [7]:
chunks = chunks_ordini + chunks_menu

In [9]:
embedder_conf = EmbedderConf(
    type="ollama",
    model="mxbai-embed-large",
)

embedder = ChunkEmbedder(embedder_conf)

# chunks = embedder.embed_chunks(chunks)

In [5]:
chunks[0].embeddings_model

'mxbai-embed-large'

In [10]:
index = faiss.IndexFlatL2(len(embedder.embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embedder.embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

doc_chunks = [
    Document(
        page_content=chunk.text, 
        metadata={
            "id":chunk.id,
            "filename": chunk.filename,
            "embeddings_model": chunk.embeddings_model
            }
    ) for chunk in chunks
]

In [11]:
vector_store.add_documents(doc_chunks)

['62b79a44-cc05-4a97-9777-4c9f0c629272',
 'e1d94796-2aee-440c-aeb2-b9f830065869',
 '67a6912f-962d-4cd1-9c63-194e46ef64ed',
 '3b276fb0-b152-4cc8-9988-3dbd59f921ea',
 '3fe4c414-6477-4c75-a1d2-3dafabd1d3af',
 '1b10a7f3-a4c0-4afb-b66c-13a1ea66ef0a',
 '9c5206de-3c78-4fc3-802d-fc84f04293db',
 '6092cd69-060d-4c3c-9f59-9b530cbb7dcf',
 '7c5bc135-7284-4285-8e62-8358bb434c58',
 'c57bd8bc-0b26-4aa4-8152-03f01b8df167',
 'b1771196-c77a-4125-a430-f5eea0bae949',
 '8ac1ab4d-6624-4d79-9cb0-42d776fc3d9f',
 'cfb6a7f3-9f36-4a07-a84b-28f28102597d',
 '25ea1a53-8776-41c5-8c7b-eb081945d0ea',
 'be25a06b-452d-4f47-9c9e-14354fe3d601',
 '81bdaea1-49de-4fe9-9005-54186ff0c354',
 '991b7753-fa97-4fcd-92aa-9aa7d390086f',
 '168e00c0-ac6e-4c75-9981-7ba406c33bdb',
 '5e18bcda-8683-4b5e-90df-da6075b2de01',
 '7e7635b6-f8ab-4811-87e1-008d55bd3733',
 '4a8e0796-9d35-4ed3-9847-5ea5152abad7',
 'd47d8b37-9329-4785-998c-cc2c5259fb99',
 '5d18dae2-f6f6-4da8-a7fe-1becbf4b96f1',
 '58359564-4f95-4ba3-ae74-8ca4d61362f1',
 '05eb86f0-2ce7-

In [12]:
retriever = vector_store.as_retriever(search_kwargs = {"k": 4})

retriever.invoke("Ordine dei naturalisti")

[Document(id='67a6912f-962d-4cd1-9c63-194e46ef64ed', metadata={'id': '739bbcaa-7ccc-5476-8716-ad3508a81cd4', 'filename': 'Manuale di Cucina.pdf', 'embeddings_model': 'mxbai-embed-large'}, page_content="<h3>🌱 Ordine dei Naturalisti</h3> Per gli adepti dell'Ordine dei Naturalisti, la cucina e molto piu che creativita: e un atto di riverenza verso la natura. Qui si celebra l'essenza primordiale di ogni ingrediente, custodendone la purezza come un prezioso segreto galattico. Nessuna trasformazione drastica, niente manipolazioni invasive: solo il “Principio di Non Alterazione”. Questo significa che se una carota vuol essere una carota, dovra restare tale, anche quando diventa arte culinaria. La sfida? Esaltare la semplicita con maestria, senza mai tradire l'identita autentica del cibo."),
 Document(id='cbec9a3e-5a9c-4c4e-b627-322c4d3188a3', metadata={'id': '3a6422b4-2ea0-5034-87fe-0f58ca3a27b7', 'filename': 'L Essenza delle Dune.pdf', 'embeddings_model': 'mxbai-embed-large'}, page_content='